# RUN THIS AFTER CONCATENATING BERT OUTPUT TO CSV'S

In [1]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import datetime

In [2]:
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# AGGREGATING USER BEHAVIOUR BY DATE

In [3]:
MERGED_TWEET_CHUNK_READ_PATH = r"/scratch/st-tlemieux-1/lfrymire/BERT-input/FINAL-INPUTS/ALL-TWEETS/COV-CT-Tweets/" #this should be a folder containing three folders - one for each type of CT tweet (make sure it ends in a slash)

AGGREGATED_TWEET_CHUNK_SAVE_PATH = r"/scratch/st-tlemieux-1/lfrymire/BERT-OUTPUT/"

In [4]:
def get_files_in_folder(folder):
    
    return [f for f in listdir(folder) if isfile(join(folder, f))]

In [5]:
chunk_paths = get_files_in_folder(MERGED_TWEET_CHUNK_READ_PATH)

In [6]:
chunk_paths = [x for x in chunk_paths if not 'TEST' in x and len(x) < 25]

In [7]:
cols_to_use = ['sentence1', 'user_id', 'label',
               'CT Tweet (Dummy)','NON-COVID CT Tweet (Dummy)', 'date', 'retweet']

bool_cols = ['label', 'CT Tweet (Dummy)','NON-COVID CT Tweet (Dummy)', 'retweet']
int_cols = ['user_ID']
# str_cols = ['Cleaned Tweet']

# obj_cols = ['language']

bool_cols_dict = {col:np.bool_ for col in bool_cols}
int_cols_dict = {col:np.int0 for col in int_cols}
# str_cols_dict = {col:str for col in str_cols}
# obj_cols_dict = {col:str for col in obj_cols}

# datatypes = {**bool_cols_dict, **str_cols_dict, **obj_cols_dict}
# datatypes = {**bool_cols_dict, **obj_cols_dict}
datatypes = {**bool_cols_dict, **int_cols_dict}

In [8]:
def get_date(s):
    return datetime.date(int(s[0:4]), int(s[5:7]), int(s[8:10]))

In [9]:
agg_dict = {'COVID-SPECIFIC CT Tweet (Dummy)':['mean', 'sum'],
            'CT Tweet (Dummy)':['mean','sum'],
            'NON-COVID CT Tweet (Dummy)':['mean','sum'],
            'retweet':['mean', 'size'],
            'Non-CT Tweet':['mean','sum']}

In [16]:
def get_missing_dates(user):
    missing_dates = np.setdiff1d(all_dates, collapsed_chunk.loc[user, 'date'].unique())    
    return missing_dates

def fill_missing_dates(collapsed_chunk):
    
    users_to_delete = []
    
    missing_dates_df_dict = {col:[] for col in collapsed_chunk.reset_index().columns}
    non_date_user_cols = [key for key in missing_dates_df_dict.keys() if not key in ('date', 'user_id')]

    for user in collapsed_chunk.index.unique():
        
        # find all missing dates
        try:
            missing_dates = get_missing_dates(user)
            
            # append np.arrays of missing values (dates, user_id, and then zeros) to dictionary
            missing_dates_df_dict['date'].append(missing_dates)
            missing_dates_df_dict['user_id'].append(np.array([user]*len(missing_dates)))
            for col in non_date_user_cols:
                missing_dates_df_dict[col].append(np.zeros(len(missing_dates)))
        except:
            users_to_delete.append(user)
            continue

    # flatten list of np arrays
    for col in missing_dates_df_dict:
        missing_dates_df_dict[col] = np.concatenate(missing_dates_df_dict[col]).ravel()

    # convert to dataframe    
    missing_dates_df = pd.DataFrame(missing_dates_df_dict)
    
    collapsed_chunk.drop(users_to_delete, inplace=True)
    
    collapsed_chunk.reset_index(inplace=True)
    
    return collapsed_chunk.append(missing_dates_df)

In [11]:
rename_dict = {('COVID-SPECIFIC CT Tweet (Dummy)', 'mean'):'COVID CT Tweet - MEAN',
                                ('COVID-SPECIFIC CT Tweet (Dummy)', 'sum'):'COVID CT Tweet - COUNT',
                                ('CT Tweet (Dummy)', 'mean'):'CT Tweet - MEAN',
                                ('CT Tweet (Dummy)', 'sum'):'CT Tweet - COUNT',
                                ('NON-COVID CT Tweet (Dummy)', 'mean'): 'NON-COVID CT Tweet - MEAN',
                                ('NON-COVID CT Tweet (Dummy)', 'sum'):'NON-COVID CT Tweet - COUNT',
                                ('retweet', 'mean'):'Retweets - MEAN',
                                ('retweet', 'size'): 'Tweet Count',
                                ('Non-CT Tweet', 'mean'):'Non-CT Tweet - MEAN',
                                ('Non-CT Tweet', 'sum'):'Non-CT Tweet - COUNT'}

In [32]:
for i, chunk_path in enumerate(chunk_paths):
    
    chunk_num = chunk_path.split('_')[-1].split('.')[0]
    print(i, chunk_num)
    
    for time_period in ('M', 'W'):

        chunk = pd.read_csv(fr"{MERGED_TWEET_CHUNK_READ_PATH}{chunk_path}", usecols=cols_to_use, dtype=datatypes, parse_dates=['date'], date_parser=get_date)

        chunk.rename(columns={'label':'COVID-SPECIFIC CT Tweet (Dummy)'}, inplace=True)
        
        # create "Non-CT Tweet" bool col
        chunk['Non-CT Tweet'] = 1 - (chunk[['COVID-SPECIFIC CT Tweet (Dummy)', 'CT Tweet (Dummy)', 'NON-COVID CT Tweet (Dummy)']].sum(axis=1) > 0)

        # MIGHT NEED 'user_id' TO BE AN INDEX TOO
        collapsed_chunk = chunk.set_index('date').groupby([pd.Grouper(freq=f'1{time_period}'), 'user_id']).agg(agg_dict)
        
        collapsed_chunk.columns = collapsed_chunk.columns.to_flat_index()        
        
        collapsed_chunk.rename(columns=rename_dict, inplace=True)
        collapsed_chunk.reset_index(inplace=True)
        collapsed_chunk.set_index('user_id', inplace=True)
        
        all_dates = collapsed_chunk['date'].unique()
        
        collapsed_chunk = fill_missing_dates(collapsed_chunk)
        # UNCOMMENT WHEN READY
        collapsed_chunk.to_csv(fr"{AGGREGATED_TWEET_CHUNK_SAVE_PATH}Chunk-{chunk_num}-AGG-BY-{time_period} - NO BERT.csv")

0 35
1 46
2 89
3 49
4 7
5 80
6 17
7 74
8 93
9 25
10 5
11 26
12 36
13 56
14 44
15 95
16 71
17 32
18 69
19 90
20 43
21 63
22 67
23 61
24 82
25 14
26 83
27 9
28 77
29 66
30 92
31 34
32 53
33 0
34 21
35 98
36 68
37 65
38 19
39 24
40 3
41 30
42 59
43 75
44 64
45 91
46 10
47 6
48 28
49 97
50 81
51 37
52 42
53 23
54 39
55 79
56 47
57 18
58 16
59 50
60 87
61 2
62 20
63 54
64 78
65 57
66 48
67 85
68 86
69 11
70 99
71 8
72 1
73 52
74 55
75 45
76 29
77 72
78 70
79 40
80 60
81 88
82 13
83 27
84 12
85 96
86 62
87 22
88 73
89 33
90 41
91 15
92 38
93 94
94 31
95 58
96 51
97 4
98 84
99 76


## MERGE ALL COLLAPSED USERS' BEHAVIOUR CHUNKS

In [33]:
month_chunk_dfs = [pd.read_csv(fr"{AGGREGATED_TWEET_CHUNK_SAVE_PATH}Chunk-{i}-AGG-BY-M - NO BERT.csv") for i, chunk_path in enumerate(chunk_paths)]
monthly_non_located_user_behaviour_df = pd.concat(month_chunk_dfs)

week_chunk_dfs = [pd.read_csv(fr"{AGGREGATED_TWEET_CHUNK_SAVE_PATH}Chunk-{i}-AGG-BY-W - NO BERT.csv") for i, chunk_path in enumerate(chunk_paths)]
weekly_non_located_user_behaviour_df = pd.concat(week_chunk_dfs)

In [35]:
len(monthly_non_located_user_behaviour_df['user_id'].unique())

82649

In [37]:
len(weekly_non_located_user_behaviour_df['user_id'].unique())

0

In [15]:
len(month_chunk_dfs[0])

21707

In [19]:
len(monthly_non_located_user_behaviour_df['user_id'].unique())

83149

In [38]:
len(monthly_non_located_user_behaviour_df) / 15

165298.0

# MERGING USER BEHAVIOUR TO GEOLOCATED USERS

In [44]:
located_users = pd.read_csv(r"/arc/project/st-tlemieux-1/data/FINAL-CLEANED-GEOLOCATED-USERS.csv")

In [48]:
monthly_final_CT_df = located_users.merge(monthly_non_located_user_behaviour_df,
                   how='left',
                   left_on='ID',
                   right_on='user_id')

weekly_final_CT_df = located_users.merge(weekly_non_located_user_behaviour_df,
                   how='left',
                   left_on='ID',
                   right_on='user_id')

### NOW SAVE TO CSV AND WE'RE DONE HERE!


In [51]:
FINAL_BEHAVIOUR_SAVE_PATH_ROOT = "/scratch/st-tlemieux-1/lfrymire/FINAL-CT-BEHAVIOUR-OUTPUT/"

In [53]:
monthly_final_CT_df

,ID,Usable FIPS,Usable STATE,Unnamed: 0,user_id,date,COVID CT Tweet - MEAN,COVID CT Tweet - COUNT,CT Tweet - MEAN,CT Tweet - COUNT,NON-COVID CT Tweet - MEAN,NON-COVID CT Tweet - COUNT,Retweets - MEAN,Tweet Count,Non-CT Tweet - MEAN,Non-CT Tweet - COUNT
0,120128474,NaN,COLORADO,93.00,120128474.00,2019-01-31,0.14,65.00,0.14,65.00,0.00,0.00,0.00,476.00,0.86,411.00
1,120128474,NaN,COLORADO,367.00,120128474.00,2019-02-28,0.11,34.00,0.11,34.00,0.00,0.00,0.00,307.00,0.89,273.00
2,120128474,NaN,COLORADO,640.00,120128474.00,2019-03-31,0.08,32.00,0.08,32.00,0.00,0.00,0.00,413.00,0.92,381.00
3,120128474,NaN,COLORADO,912.00,120128474.00,2019-04-30,0.12,40.00,0.12,40.00,0.00,0.00,0.00,320.00,0.88,280.00
4,120128474,NaN,COLORADO,1185.00,120128474.00,2019-05-31,0.07,32.00,0.07,32.00,0.00,0.00,0.00,439.00,0.93,407.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2473210,170843799,20091.00,KANSAS,19780.00,170843799.00,2021-02-28,0.00,0.00,0.00,0.00,0.00,0.00,0.00,43.00,1.00,43.00
2473211,170843799,20091.00,KANSAS,20549.00,170843799.00,2021-03-31,0.00,0.00,0.00,0.00,0.00,0.00,0.00,47.00,1.00,47.00
2473212,170843799,20091.00,KANSAS,21311.00,170843799.00,2021-04-30,0.00,0.00,0.00,0.00,0.00,0.00,0.00,40.00,1.00,40.00
2473213,170843799,20091.00,KANSAS,22060.00,170843799.00,2021-05-31,0.00,0.00,0.02,1.00,0.02,1.00,0.00,49.00,0.98,48.00


In [55]:
weekly_final_CT_df.memory_usage().sum()

1444070984

In [52]:
monthly_final_CT_df.to_csv(f"{FINAL_BEHAVIOUR_SAVE_PATH_ROOT}Monthly Behaviour - NO BERT.csv")

weekly_final_CT_df.to_csv(f"{FINAL_BEHAVIOUR_SAVE_PATH_ROOT}Weekyl Behaviour - NO BERT.csv")